In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import time
import datetime

In [2]:
read_dir = './data'
csv_files = os.listdir(read_dir)

In [3]:
load_dfs = []
ren_dfs = []
for csv_file in csv_files:
    if 'REN' not in csv_file:
        cur_df = pd.read_csv(
            os.path.join(read_dir, csv_file),
            parse_dates={'start_dt': ['INTERVALSTARTTIME_GMT'], 'end_dt': ['INTERVALENDTIME_GMT']}
        )
        load_dfs.append(cur_df)
    else:
        cur_df = pd.read_csv(
            os.path.join(read_dir, csv_file),
            parse_dates={'start_dt': ['INTERVALSTARTTIME_GMT'], 'end_dt': ['INTERVALENDTIME_GMT']}
        )
        ren_dfs.append(cur_df)

load_df = pd.concat(load_dfs)
ren_df = pd.concat(ren_dfs)

In [4]:
load_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397850 entries, 0 to 11814
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   start_dt        397850 non-null  datetime64[ns, UTC]
 1   end_dt          397850 non-null  datetime64[ns, UTC]
 2   LOAD_TYPE       397850 non-null  int64              
 3   OPR_DT          397850 non-null  object             
 4   OPR_HR          397850 non-null  int64              
 5   OPR_INTERVAL    397850 non-null  int64              
 6   MARKET_RUN_ID   397850 non-null  object             
 7   TAC_AREA_NAME   397850 non-null  object             
 8   LABEL           397850 non-null  object             
 9   XML_DATA_ITEM   397850 non-null  object             
 10  POS             397850 non-null  float64            
 11  MW              397850 non-null  int64              
 12  EXECUTION_TYPE  397850 non-null  object             
 13  GROUP          

In [5]:
ren_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141160 entries, 0 to 3594
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   start_dt           141160 non-null  datetime64[ns, UTC]
 1   end_dt             141160 non-null  datetime64[ns, UTC]
 2   OPR_DT             141160 non-null  object             
 3   OPR_HR             141160 non-null  int64              
 4   OPR_INTERVAL       141160 non-null  int64              
 5   TRADING_HUB        141160 non-null  object             
 6   RENEWABLE_TYPE     141160 non-null  object             
 7   LABEL              141160 non-null  object             
 8   XML_DATA_ITEM      141160 non-null  object             
 9   MARKET_RUN_ID_POS  141160 non-null  int64              
 10  RENEW_POS          141160 non-null  int64              
 11  MW                 141160 non-null  float64            
 12  MARKET_RUN_ID      141160 non-nu

In [6]:
load_df = load_df.drop(columns=['OPR_DT',
                                'OPR_HR',
                                'OPR_INTERVAL',
                                'LOAD_TYPE',
                                'MARKET_RUN_ID',
                                'XML_DATA_ITEM',
                                'POS',
                                'EXECUTION_TYPE',
                                'GROUP'
                               ])

In [7]:
ren_df = ren_df.drop(columns=['OPR_DT',
                              'OPR_HR',
                              'OPR_INTERVAL',
                              'XML_DATA_ITEM',
                              'MARKET_RUN_ID_POS',
                              'RENEW_POS',
                              'MARKET_RUN_ID',
                              'GROUP'
                             ])

In [8]:
# clean the integrated load df

In [9]:
cur_load_tac = 'CA ISO-TAC'

In [10]:
load_df = load_df[load_df.TAC_AREA_NAME == cur_load_tac]

In [11]:
load_df = load_df.drop(columns=['TAC_AREA_NAME',
                                'LABEL'
                               ])

In [12]:
load_df = load_df.rename(columns={'MW': 'total_integrated_load_MW'})

In [13]:
load_df.head()

,start_dt,end_dt,total_integrated_load_MW
21,2019-06-01 01:00:00+00:00,2019-06-01 02:00:00+00:00,27212
22,2019-06-01 05:00:00+00:00,2019-06-01 06:00:00+00:00,25314
23,2019-06-01 00:00:00+00:00,2019-06-01 01:00:00+00:00,26682
24,2019-06-01 02:00:00+00:00,2019-06-01 03:00:00+00:00,27380
25,2019-06-01 03:00:00+00:00,2019-06-01 04:00:00+00:00,27734


In [14]:
load_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28256 entries, 21 to 11606
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   start_dt                  28256 non-null  datetime64[ns, UTC]
 1   end_dt                    28256 non-null  datetime64[ns, UTC]
 2   total_integrated_load_MW  28256 non-null  int64              
dtypes: datetime64[ns, UTC](2), int64(1)
memory usage: 883.0 KB


In [15]:
# clean the renewable generation df

In [16]:
ren_df = ren_df.groupby(['start_dt', 'RENEWABLE_TYPE'])['MW'].aggregate('sum').unstack()

In [17]:
ren_df.head()

RENEWABLE_TYPE,Solar,Wind
start_dt,,
2017-07-01 07:00:00+00:00,-31.09459,2881.98320
2017-07-01 08:00:00+00:00,-30.83811,2765.90229
2017-07-01 09:00:00+00:00,-30.48083,2791.72076
2017-07-01 10:00:00+00:00,-31.98818,2549.59684
2017-07-01 11:00:00+00:00,-32.67630,2502.67552


In [18]:
ren_df.columns

Index(['Solar', 'Wind'], dtype='object', name='RENEWABLE_TYPE')

In [19]:
ren_df = ren_df.rename(columns={'Solar': 'solar_generation_MW', 'Wind': 'wind_generation_MW'})

In [20]:
ren_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28232 entries, 2017-07-01 07:00:00+00:00 to 2020-10-30 22:00:00+00:00
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   solar_generation_MW  28232 non-null  float64
 1   wind_generation_MW   28232 non-null  float64
dtypes: float64(2)
memory usage: 661.7 KB


In [21]:
df = load_df.merge(ren_df, how='inner', on='start_dt').sort_values(by=['start_dt']).reset_index(drop=True)

In [22]:
df.head()

,start_dt,end_dt,total_integrated_load_MW,solar_generation_MW,wind_generation_MW
0,2017-07-01 07:00:00+00:00,2017-07-01 08:00:00+00:00,25703,-31.09459,2881.98320
1,2017-07-01 08:00:00+00:00,2017-07-01 09:00:00+00:00,24043,-30.83811,2765.90229
2,2017-07-01 09:00:00+00:00,2017-07-01 10:00:00+00:00,22988,-30.48083,2791.72076
3,2017-07-01 10:00:00+00:00,2017-07-01 11:00:00+00:00,22423,-31.98818,2549.59684
4,2017-07-01 11:00:00+00:00,2017-07-01 12:00:00+00:00,22235,-32.67630,2502.67552


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28232 entries, 0 to 28231
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   start_dt                  28232 non-null  datetime64[ns, UTC]
 1   end_dt                    28232 non-null  datetime64[ns, UTC]
 2   total_integrated_load_MW  28232 non-null  int64              
 3   solar_generation_MW       28232 non-null  float64            
 4   wind_generation_MW        28232 non-null  float64            
dtypes: datetime64[ns, UTC](2), float64(2), int64(1)
memory usage: 1.1 MB


In [24]:
df.tail()

,start_dt,end_dt,total_integrated_load_MW,solar_generation_MW,wind_generation_MW
28227,2020-10-30 18:00:00+00:00,2020-10-30 19:00:00+00:00,23224,8243.72168,148.89958
28228,2020-10-30 19:00:00+00:00,2020-10-30 20:00:00+00:00,23304,8151.44188,150.94192
28229,2020-10-30 20:00:00+00:00,2020-10-30 21:00:00+00:00,23561,8328.81216,151.05136
28230,2020-10-30 21:00:00+00:00,2020-10-30 22:00:00+00:00,24153,8381.43917,161.57291
28231,2020-10-30 22:00:00+00:00,2020-10-30 23:00:00+00:00,24810,7707.46670,135.37680


In [25]:
df.to_csv('../data/CAISO-20170701-20201030.csv')

In [ ]:
df = pd.read_csv('../data/CAISO-20170701-20201030.csv')
df['start_dt'] = pd.to_datetime(df['start_dt'])
df['end_dt'] = pd.to_datetime(df['end_dt'])
df['covariate_x'] = df['end_dt'].apply(lambda i:[i.week,i.dayofweek,i.hour])
df['ramp'] = df['total_integrated_load_MW'] - df['solar_generation_MW'] - df['wind_generation_MW']
df.to_csv('../data/CAISO-20170701-20201030.csv',index=False)
df.head()